In [2]:
import json
import pandas
import numpy
import os
import sys
from os import path

In [82]:
company_name = 'Royal Apothecary'

inventory_dates = [
    '09/30/2020',
    '10/31/2020',
    '11/30/2020',
    '12/31/2020',
    '01/31/2021',
    '02/28/2021',
    '03/31/2021',
    '04/30/2021',
    '05/31/2021',
    '06/30/2021',
    '07/31/2021',
    '08/31/2021',
    '09/30/2021',
]
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
%autoreload 2

sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts/analysis")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

from util import active_inventory_util as util

d = util.Download(
    incoming_files=[
        'data/20210930/royal_apothecary_incoming_transfer_packages_20200101_20210930.xlsx',
    ],
    outgoing_files=[
        'data/20210930/royal_apothecary_outgoing_transfer_packages_20210101_20211006.xlsx',
    ],
#     packages_files=[],
    sales_transactions_files=[
        'data/20210930/royal_apothecary_sales_transactions_20200101_20201231.xlsx',
        'data/20210930/royal_apothecary_sales_transactions_20210101_20201006.xlsx',
    ],
)

Opened file data/20210930/royal_apothecary_incoming_transfer_packages_20200101_20210930.xlsx with 26 columns and 5484 rows
Dataframe columns:
Index(['transfer_row_id', 'delivery_row_id', 'package_row_id', 'delivery_type',
       'license_number', 'manifest_number', 'created_date',
       'transfer_lab_results_status', 'received_datetime',
       'shipper_facility_license_number', 'shipper_facility_name',
       'recipient_facility_license_number', 'recipient_facility_name',
       'shipment_type_name', 'shipment_transaction_type', 'package_id',
       'package_label', 'type', 'product_category_name', 'product_name',
       'shipper_wholesale_price', 'shipped_quantity',
       'package_lab_results_status', 'shipment_package_state',
       'is_testing_sample', 'is_trade_sample'],
      dtype='object')
Opened file data/20210930/royal_apothecary_outgoing_transfer_packages_20210101_20211006.xlsx with 20 columns and 15 rows
Dataframe columns:
Index(['transfer_row_id', 'delivery_row_id', 'pac

In [84]:
q = util.Query()
q.inventory_dates = inventory_dates
q.company_name = company_name

id_to_history = util.get_histories(d)
util.print_counts(id_to_history)
util.create_inventory_xlsx(id_to_history, q)

Only outgoing: 7
Only incoming: 1487
In and out: 8
In and sold at least once 3950
In and sold many times 3792
Total pkgs: 5475
Wrote result to out/Royal Apothecary_inventory_by_month.xls
Excluded 71 / 5475 packages from consideration (1.30%)
  MANY_INCOMING: 40 times
  MISSING_INCOMING: 31 times


In [85]:
q = util.Query()
q.inventory_dates = inventory_dates
q.company_name = company_name

id_to_history = util.get_histories(d)
util.print_counts(id_to_history)
date_to_inventory_records = util.create_inventory_dataframes(id_to_history, q)
date_to_inventory_records.keys()

Only outgoing: 7
Only incoming: 1487
In and out: 8
In and sold at least once 3950
In and sold many times 3792
Total pkgs: 5475


dict_keys(['09/30/2020', '10/31/2020', '11/30/2020', '12/31/2020', '01/31/2021', '02/28/2021', '03/31/2021', '04/30/2021', '05/31/2021', '06/30/2021', '07/31/2021', '08/31/2021', '09/30/2021'])

In [86]:
date_to_inventory_dataframe = {}
for date, inventory_records in date_to_inventory_records.items():
    date_to_inventory_dataframe[date] = pandas.DataFrame(
        inventory_records,
        columns=[
            'package_id',
            'Arrived Date',
            'Product Category',
            'Product Name',
            'Current Quantity',
            'Sold Date',
        ]
    )
    
date_to_inventory_dataframe[list(date_to_inventory_dataframe.keys())[0]]

,package_id,Arrived Date,Product Category,Product Name,Current Quantity,Sold Date
0,8284414,09/30/2020,Flower (packaged eighth - each),3Cs Strawbinati 3.5 jar,0,10/09/2020
1,8269529,09/29/2020,Extract (weight - each),Pink Banana Cake Badder (Gram Jar),20,
2,8275266,09/29/2020,Vape Cartridge (volume - each),13871 Raw Garden Cartridge 1.0g Indica Mendo P...,40,
3,8275261,09/29/2020,Vape Cartridge (volume - each),11294 Raw Garden Cartridge 1.0g Indica Skydogg...,40,
4,8275262,09/29/2020,Vape Cartridge (volume - each),13684 Raw Garden Cartridge 1.0g 1:1 CBD/THC Ar...,20,
...,...,...,...,...,...,...
1459,2975297,02/12/2020,Extract (volume - each),Acapulco Gold Vape Cart,1,
1460,2975671,02/12/2020,Extract (weight - each),831 Extracts Live Resin Sugar Gelato,1,
1461,2609323,02/12/2020,Pre-Roll Infused,AQ Infused PreRoll,4,
1462,2578564,02/12/2020,Flower,THC D- Domo G Indica 3.5 g Jar,7,


In [87]:
raw_incoming_transfer_packages_dataframe = pandas.read_excel('data/20210930/royal_apothecary_incoming_transfer_packages_20200101_20210930.xlsx', header=0)
len(raw_incoming_transfer_packages_dataframe.index), raw_incoming_transfer_packages_dataframe.columns

/Users/warren/venvs/bespoke/lib/python3.7/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(5484,
 Index(['transfer_row_id', 'delivery_row_id', 'package_row_id', 'delivery_type',
        'license_number', 'manifest_number', 'created_date',
        'transfer_lab_results_status', 'received_datetime',
        'shipper_facility_license_number', 'shipper_facility_name',
        'recipient_facility_license_number', 'recipient_facility_name',
        'shipment_type_name', 'shipment_transaction_type', 'package_id',
        'package_label', 'type', 'product_category_name', 'product_name',
        'shipper_wholesale_price', 'shipped_quantity',
        'package_lab_results_status', 'shipment_package_state',
        'is_testing_sample', 'is_trade_sample'],
       dtype='object'))

In [88]:
incoming_transfer_packages_dataframe = raw_incoming_transfer_packages_dataframe[[
#     'date_type',
#     'transfer_row_id',
#     'delivery_row_id',
#     'package_row_id',
    'delivery_type',
    'manifest_number',
    'created_date',
    'received_datetime',
    'shipper_facility_license_number',
    'shipper_facility_name',
    'recipient_facility_license_number',
    'recipient_facility_name',
    'shipment_type_name',
    'shipment_transaction_type',
    'package_id',
    'package_label',
    'type',
    'product_category_name',
    'product_name',
    'shipper_wholesale_price',
    'shipped_quantity',
    'package_lab_results_status',
    'shipment_package_state',
    'is_testing_sample',
    'is_trade_sample'
]]

In [93]:
for date, inventory_dataframe in date_to_inventory_dataframe.items():
    print(date)
    print(f'# of packages in inventory: {len(inventory_dataframe.index)}')

    inventory_with_incoming_transfer_packages_dataframe = inventory_dataframe.astype({'package_id': 'int64'}).merge(incoming_transfer_packages_dataframe, on='package_id', how='inner', suffixes=('_l', '_r'))
#     print(f'# of packages in inventory with incoming package: {len(inventory_with_incoming_transfer_packages_dataframe.index)}')
    
    inventory_with_cost_records = inventory_with_incoming_transfer_packages_dataframe.to_dict('record')

    total_valuation_cost = 0
    import math
    for inventory_with_cost_record in inventory_with_cost_records:
        incoming_shipped_price = inventory_with_cost_record['shipper_wholesale_price']
        if math.isnan(incoming_shipped_price):
            incoming_shipped_price = 0
        incoming_shipped_quantity = inventory_with_cost_record['shipped_quantity']
        current_quantity = inventory_with_cost_record['Current Quantity']
        total_valuation_cost += float(current_quantity) * (incoming_shipped_price / incoming_shipped_quantity)

    print(f'Inventory valuation (based on COST): ${total_valuation_cost}')
    print('')

09/30/2020
# of packages in inventory: 1464
Inventory valuation (based on COST): $164802.64571975905

10/31/2020
# of packages in inventory: 1713
Inventory valuation (based on COST): $214048.16463415074

11/30/2020
# of packages in inventory: 1671
Inventory valuation (based on COST): $160038.30420620358

12/31/2020
# of packages in inventory: 1813
Inventory valuation (based on COST): $148865.83005506246

01/31/2021
# of packages in inventory: 2070
Inventory valuation (based on COST): $165997.68341993817

02/28/2021
# of packages in inventory: 2232
Inventory valuation (based on COST): $152399.19214778862

03/31/2021
# of packages in inventory: 2389
Inventory valuation (based on COST): $165568.29958287667

04/30/2021
# of packages in inventory: 2516
Inventory valuation (based on COST): $198215.88367231708

05/31/2021
# of packages in inventory: 2644
Inventory valuation (based on COST): $205491.10586772586

06/30/2021
# of packages in inventory: 2824
Inventory valuation (based on COST): $